**LLM모델(genai)을 이용한 피파온라인 유저의 감성분석**

In [7]:
import pandas as pd
import numpy as np
import pathlib
import textwrap
import time
import google.generativeai as genai
from google.colab import userdata
from IPython.display import display
from IPython.display import Markdown

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [9]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [10]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.0-flash-exp
models/gemini-exp-1206
models/gemini-exp-1121
models/gemini-exp-1114
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/learnlm-1.5-pro-experimental


In [11]:
# 텍스트 전용
model = genai.GenerativeModel('gemini-1.5-flash')

In [12]:
file_path = "/content/drive/My Drive/fifa/dataset1.csv"

In [13]:
fifa_df = pd.read_csv(file_path)
fifa_df.head()

,date,community,postId,title,body,viewCount,commentCount
0,2020-07-01,인벤,16258,스몰링 사재기 갑니다.jpg,3억5천바라봅니다,227,11
1,2020-07-01,인벤,36379,현재까지 빠칭코 출시 시기 모음.,"토티, 토츠 출시 1주일 후에 전부다 빠짐없이 빠찡코 (럭키상점) 등장100퍼센트는...",192,12
2,2020-07-01,인벤,28187,좀전에 3바란뜬사람,피파하면서 비싼거 강화 절대안하는데 4강까지 갓으니 걍 해보렵니다ㅠ,116,22
3,2020-07-01,인벤,28331,주공 날강두 은카 결과,감사합니다 인벤행님들착하게 살게요 ㅋㅋ,132,11
4,2020-07-01,에펨코,1392,1277 은카 파시는 선생님,저는 일단 자보겠읍니다.댓글로 남겼지만팩트 하나 말씀드리자면오버롤 107로 개높은 ...,129,17


In [46]:
def make_review_pn(review):
  prompt = f'''
  아래 리뷰를 읽고 긍정인지 부정, 알려줘 그리고 만약 긍정이면 1을 반환 부정이면 0을 반환해줘 예를들어 "1", "0"
  리뷰:{review}
  '''
  response = model.generate_content(prompt)

  return response.text

In [47]:
# 10개의 행만 감정분석
df_review_10 = fifa_df.iloc[:10, :]
df_review_10

,date,community,postId,title,body,viewCount,commentCount
0,2020-07-01,인벤,16258,스몰링 사재기 갑니다.jpg,3억5천바라봅니다,227,11
1,2020-07-01,인벤,36379,현재까지 빠칭코 출시 시기 모음.,"토티, 토츠 출시 1주일 후에 전부다 빠짐없이 빠찡코 (럭키상점) 등장100퍼센트는...",192,12
2,2020-07-01,인벤,28187,좀전에 3바란뜬사람,피파하면서 비싼거 강화 절대안하는데 4강까지 갓으니 걍 해보렵니다ㅠ,116,22
3,2020-07-01,인벤,28331,주공 날강두 은카 결과,감사합니다 인벤행님들착하게 살게요 ㅋㅋ,132,11
4,2020-07-01,에펨코,1392,1277 은카 파시는 선생님,저는 일단 자보겠읍니다.댓글로 남겼지만팩트 하나 말씀드리자면오버롤 107로 개높은 ...,129,17
5,2020-07-01,인벤,1340,110억짜리 ZD.gif,오늘도 나지막이 읊어봅니다.에메에에에에에에에에에에에에에에에에에에에에에에에에에에에에에...,158,18
6,2020-07-01,인벤,23358,요즘 잠잠한 피벤 1대장 근황.JPG,한 때 어그로 명단 작성 파동으로 피벤에 큰 분란을 일으켰던 나XX.요즘 근황을 알...,175,15
7,2020-07-01,인벤,3759,강화이벤트 못 기다리고 강화질렀습니다,챔시즌 나왔을 때 강화로 너무 많이 해먹어서그 이후로 오랫동안 본섭에서 강화는 손도...,106,11
8,2020-07-01,인벤,2006,20토츠 호날두 5카 후기,아직 많이 사용해보진 못했는데 당연히 헤더좋고 몸싸움 좋은데풀캐미 슛파워 122 중...,107,12
9,2020-07-01,에펨코,22150,와.... 형들 에이전트 무조건 하세요,접을 마음으로 팀 다 팔고 돌렸는데.... 다시 해야겠네요,114,11


In [48]:
df_review_10.loc[:, '긍정/부정'] = df_review_10['body'].apply(make_review_pn)

<ipython-input-48-14c309769c21>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_review_10.loc[:, '긍정/부정'] = df_review_10['body'].apply(make_review_pn)


In [49]:
df_review_10['긍정/부정'].head()

# 줄바꿈 삭제
df_review_10.loc[:, '긍정/부정'] = df_review_10['긍정/부정'].apply(lambda x: x.replace("\n", ""))

In [50]:
df_review_10.head()

,date,community,postId,title,body,viewCount,commentCount,긍정/부정
0,2020-07-01,인벤,16258,스몰링 사재기 갑니다.jpg,3억5천바라봅니다,227,11,0
1,2020-07-01,인벤,36379,현재까지 빠칭코 출시 시기 모음.,"토티, 토츠 출시 1주일 후에 전부다 빠짐없이 빠찡코 (럭키상점) 등장100퍼센트는...",192,12,0
2,2020-07-01,인벤,28187,좀전에 3바란뜬사람,피파하면서 비싼거 강화 절대안하는데 4강까지 갓으니 걍 해보렵니다ㅠ,116,22,1
3,2020-07-01,인벤,28331,주공 날강두 은카 결과,감사합니다 인벤행님들착하게 살게요 ㅋㅋ,132,11,1
4,2020-07-01,에펨코,1392,1277 은카 파시는 선생님,저는 일단 자보겠읍니다.댓글로 남겼지만팩트 하나 말씀드리자면오버롤 107로 개높은 ...,129,17,0


In [32]:
# 30행 씩 처리
df_review_30 = fifa_df.loc[:30:]
df_review_30

,date,community,postId,title,body,viewCount,commentCount
0,2020-07-01,인벤,16258,스몰링 사재기 갑니다.jpg,3억5천바라봅니다,227,11
1,2020-07-01,인벤,36379,현재까지 빠칭코 출시 시기 모음.,"토티, 토츠 출시 1주일 후에 전부다 빠짐없이 빠찡코 (럭키상점) 등장100퍼센트는...",192,12
2,2020-07-01,인벤,28187,좀전에 3바란뜬사람,피파하면서 비싼거 강화 절대안하는데 4강까지 갓으니 걍 해보렵니다ㅠ,116,22
3,2020-07-01,인벤,28331,주공 날강두 은카 결과,감사합니다 인벤행님들착하게 살게요 ㅋㅋ,132,11
4,2020-07-01,에펨코,1392,1277 은카 파시는 선생님,저는 일단 자보겠읍니다.댓글로 남겼지만팩트 하나 말씀드리자면오버롤 107로 개높은 ...,129,17
5,2020-07-01,인벤,1340,110억짜리 ZD.gif,오늘도 나지막이 읊어봅니다.에메에에에에에에에에에에에에에에에에에에에에에에에에에에에에에...,158,18
6,2020-07-01,인벤,23358,요즘 잠잠한 피벤 1대장 근황.JPG,한 때 어그로 명단 작성 파동으로 피벤에 큰 분란을 일으켰던 나XX.요즘 근황을 알...,175,15
7,2020-07-01,인벤,3759,강화이벤트 못 기다리고 강화질렀습니다,챔시즌 나왔을 때 강화로 너무 많이 해먹어서그 이후로 오랫동안 본섭에서 강화는 손도...,106,11
8,2020-07-01,인벤,2006,20토츠 호날두 5카 후기,아직 많이 사용해보진 못했는데 당연히 헤더좋고 몸싸움 좋은데풀캐미 슛파워 122 중...,107,12
9,2020-07-01,에펨코,22150,와.... 형들 에이전트 무조건 하세요,접을 마음으로 팀 다 팔고 돌렸는데.... 다시 해야겠네요,114,11


In [37]:
# 1분에 15개씩 처리
def process_reviews_in_batches(df_review_30, batch_size=15, delay=65):
    total_rows = len(df_review_30)
    for start_idx in range(0, total_rows, batch_size):
        end_idx = min(start_idx + batch_size, total_rows)
        batch = df_review_30.loc[start_idx:end_idx-1, :].copy()

        # make_review_pn 함수를 적용
        batch['긍정/부정'] = batch['body'].apply(make_review_pn)
        # 처리된 배치를 원본 데이터프레임에 업데이트
        df_review_30.loc[start_idx:end_idx-1, '긍정/부정'] = batch['긍정/부정'].values
        print(df_review_30["긍정/부정"])

        # 더 처리할 행이 남아있다면 지정된 지연 시간만큼 대기
        if end_idx < total_rows:
            time.sleep(delay)  # 지정된 시간(초) 동안 대기

    return df_review_30

# 1분에 15개씩 나눠서 처리
processed_df = process_reviews_in_batches(df_review_30)

# 30행까지 2분

ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 907.84ms


0     긍정\n
1     부정\n
2     혼합\n
3     긍정\n
4     혼합\n
5     긍정\n
6     부정\n
7     긍정\n
8     긍정\n
9     부정\n
10     NaN
11     NaN
12     NaN
13     NaN
14     NaN
15     NaN
16     NaN
17     NaN
18     NaN
19     NaN
20     NaN
21     NaN
22     NaN
23     NaN
24     NaN
25     NaN
26     NaN
27     NaN
28     NaN
29     NaN
30    혼합\n
Name: 긍정/부정, dtype: object
0     긍정\n
1     부정\n
2     혼합\n
3     긍정\n
4     혼합\n
5     긍정\n
6     부정\n
7     긍정\n
8     긍정\n
9     부정\n
10    부정\n
11    부정\n
12    긍정\n
13    긍정\n
14    부정\n
15    혼합\n
16    혼합\n
17    혼합\n
18    긍정\n
19    혼합\n
20     NaN
21     NaN
22     NaN
23     NaN
24     NaN
25     NaN
26     NaN
27     NaN
28     NaN
29     NaN
30    혼합\n
Name: 긍정/부정, dtype: object


ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 907.47ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 1082.42ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 806.71ms


0     긍정\n
1     부정\n
2     혼합\n
3     긍정\n
4     혼합\n
5     긍정\n
6     부정\n
7     긍정\n
8     긍정\n
9     부정\n
10    부정\n
11    부정\n
12    긍정\n
13    긍정\n
14    부정\n
15    혼합\n
16    혼합\n
17    혼합\n
18    긍정\n
19    혼합\n
20    긍정\n
21    부정\n
22    부정\n
23    긍정\n
24    부정\n
25    긍정\n
26    혼합\n
27    부정\n
28    긍정\n
29    혼합\n
30    혼합\n
Name: 긍정/부정, dtype: object
0     긍정\n
1     부정\n
2     혼합\n
3     긍정\n
4     혼합\n
5     긍정\n
6     부정\n
7     긍정\n
8     긍정\n
9     부정\n
10    부정\n
11    부정\n
12    긍정\n
13    긍정\n
14    부정\n
15    혼합\n
16    혼합\n
17    혼합\n
18    긍정\n
19    혼합\n
20    긍정\n
21    부정\n
22    부정\n
23    긍정\n
24    부정\n
25    긍정\n
26    혼합\n
27    부정\n
28    긍정\n
29    혼합\n
30    혼합\n
Name: 긍정/부정, dtype: object


In [40]:
processed_df['긍정/부정'] = processed_df['긍정/부정'].apply(lambda x : x.replace("\n",""))
processed_df

<ipython-input-40-b8dbad2ad882>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_df['긍정/부정'] = processed_df['긍정/부정'].apply(lambda x : x.replace("\n",""))


,date,community,postId,title,body,viewCount,commentCount,긍정/부정
0,2020-07-01,인벤,16258,스몰링 사재기 갑니다.jpg,3억5천바라봅니다,227,11,긍정
1,2020-07-01,인벤,36379,현재까지 빠칭코 출시 시기 모음.,"토티, 토츠 출시 1주일 후에 전부다 빠짐없이 빠찡코 (럭키상점) 등장100퍼센트는...",192,12,부정
2,2020-07-01,인벤,28187,좀전에 3바란뜬사람,피파하면서 비싼거 강화 절대안하는데 4강까지 갓으니 걍 해보렵니다ㅠ,116,22,혼합
3,2020-07-01,인벤,28331,주공 날강두 은카 결과,감사합니다 인벤행님들착하게 살게요 ㅋㅋ,132,11,긍정
4,2020-07-01,에펨코,1392,1277 은카 파시는 선생님,저는 일단 자보겠읍니다.댓글로 남겼지만팩트 하나 말씀드리자면오버롤 107로 개높은 ...,129,17,혼합
5,2020-07-01,인벤,1340,110억짜리 ZD.gif,오늘도 나지막이 읊어봅니다.에메에에에에에에에에에에에에에에에에에에에에에에에에에에에에에...,158,18,긍정
6,2020-07-01,인벤,23358,요즘 잠잠한 피벤 1대장 근황.JPG,한 때 어그로 명단 작성 파동으로 피벤에 큰 분란을 일으켰던 나XX.요즘 근황을 알...,175,15,부정
7,2020-07-01,인벤,3759,강화이벤트 못 기다리고 강화질렀습니다,챔시즌 나왔을 때 강화로 너무 많이 해먹어서그 이후로 오랫동안 본섭에서 강화는 손도...,106,11,긍정
8,2020-07-01,인벤,2006,20토츠 호날두 5카 후기,아직 많이 사용해보진 못했는데 당연히 헤더좋고 몸싸움 좋은데풀캐미 슛파워 122 중...,107,12,긍정
9,2020-07-01,에펨코,22150,와.... 형들 에이전트 무조건 하세요,접을 마음으로 팀 다 팔고 돌렸는데.... 다시 해야겠네요,114,11,부정
